In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [28]:
# Cargar los datos de juegos (descripciones)
df_juegos = pd.read_csv('../data_extraction/boardgames_10000_juegos.csv')

In [29]:
df_juegos.columns

Index(['BGGId', 'Name', 'Year_Published', 'Description', 'Min_Players',
       'Max_Players', 'Min_Playtime', 'Max_Playtime', 'Average_Rating',
       'Bayesian_Average_Rating', 'Number_of_Ratings', 'Mechanics',
       'Categories'],
      dtype='object')

In [30]:
# Obtener la descripción de los juegos
df_descripciones = df_juegos[['BGGId','Name','Description', 'Mechanics', 'Categories', 'Average_Rating', 'Bayesian_Average_Rating']]  

In [31]:
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    10000 non-null  int64  
 1   Name                     10000 non-null  object 
 2   Description              9992 non-null   object 
 3   Mechanics                10000 non-null  object 
 4   Categories               10000 non-null  object 
 5   Average_Rating           10000 non-null  float64
 6   Bayesian_Average_Rating  10000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 547.0+ KB


In [32]:
df_descripciones.dropna(inplace=True)
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    9992 non-null   int64  
 1   Name                     9992 non-null   object 
 2   Description              9992 non-null   object 
 3   Mechanics                9992 non-null   object 
 4   Categories               9992 non-null   object 
 5   Average_Rating           9992 non-null   float64
 6   Bayesian_Average_Rating  9992 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 624.5+ KB


C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_26012\1111501472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones.dropna(inplace=True)


In [33]:
df_descripciones.sample(5)

,BGGId,Name,Description,Mechanics,Categories,Average_Rating,Bayesian_Average_Rating
1807,62075,It Came From the Sky,From the Publisher:&#10;&#10;A Living jungle a...,[],[],6.0000,0.0
9504,165290,GRID Autosport,Grid Autosport (styled as GRID Autosport) is a...,[],[],7.8375,0.0
282,188290,Alarums & Excursions (Issue 72 - Aug 1981),Alarums & Excursions&#10;Issue 72&#10;August 1...,[],[],0.0000,0.0
6005,86636,The Three Rings of Cassia,From publisher blurb:&#10;&#10;Eternally spinn...,[],[],0.0000,0.0
4002,61507,Dire Heroes: Gas Attack at Ypres,"The Ypres salient, a 6 mile (10 km) jut into G...","['Area Majority / Influence', 'Dice Rolling', ...","['Book', 'Expansion for Base-game', 'Wargame',...",7.7500,0.0


In [34]:
# Asegúrate de descargar stopwords y wordnet de nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Función mejorada de preprocesamiento de texto
def preprocesar_descripcion(texto):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Eliminar entidades HTML y otros caracteres no deseados
    texto = re.sub(r'&#?\w+;', ' ', texto)  # Reemplazar entidades HTML como &#10; 
    texto = re.sub(r'\W+', ' ', texto)  # Eliminar caracteres no alfanuméricos
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    
    # Convertir a minúsculas, eliminar stopwords y lematizar (solo si el texto es válido)
    if isinstance(texto, str):
        palabras = [lemmatizer.lemmatize(palabra) for palabra in texto.lower().split() if palabra not in stop_words]
        return " ".join(palabras)
    else:
        return ""  # Devolver cadena vacía si no es un texto válido

# Aplicar el preprocesamiento mejorado a las descripciones
df_descripciones['Descripcion_limpia'] = df_descripciones['Description'].apply(preprocesar_descripcion)


[nltk_data] Downloading package stopwords to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_26012\3183920906.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones['Descripcion_limpia'] = df_descripciones['Description'].apply(preprocesar_descripcion)


In [35]:
# Convertir listas de mecánicas y categorías en cadenas de texto
df_descripciones['Mecanicas'] = df_descripciones['Mechanics'].apply(lambda x: ', '.join(x))
df_descripciones['Categorias'] = df_descripciones['Categories'].apply(lambda x: ', '.join(x))

# Combinar la descripción limpia con las mecánicas y categorías
df_descripciones['Descripcion_completa'] = df_descripciones['Descripcion_limpia'] + " " + df_descripciones['Mecanicas'] + " " + df_descripciones['Categorias']

C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_26012\2397482331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones['Mecanicas'] = df_descripciones['Mechanics'].apply(lambda x: ', '.join(x))
C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_26012\2397482331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones['Categorias'] = df_descripciones['Categories'].apply(lambda x: ', '.join(x))
C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_26012\2397482331.py:6: SettingWithCopy

In [36]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Entrenar Doc2Vec usando los índices del DataFrame como etiquetas
documents = [TaggedDocument(doc.split(), [idx]) for idx, doc in zip(df_descripciones.index, df_descripciones['Descripcion_completa'])]

# Entrenar el modelo Doc2Vec
model = Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4)


In [37]:
# Función para recomendar juegos usando Doc2Vec y similaridad del coseno
def recomendar_juegos_doc2vec(prompt, df_descripciones, model, top_n=5):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos
    vectors = [model.dv[i] for i in range(len(df_descripciones))]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Obtener los índices de los juegos con mayor similaridad
    indices_recomendados = cosine_similarities.argsort()[-top_n:][::-1]

    # Mostrar los juegos recomendados
    print("Juegos recomendados:")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad: {cosine_similarities[index]:.4f})")

In [38]:
# Prompt del usuario
prompt_usuario = 'I am looking for a game that is great for solo play and lasts about 30 minutes. I love games with a rich storytelling experience and immersive themes.'
# Ejecutar la recomendación usando Doc2Vec
recomendar_juegos_doc2vec(prompt_usuario, df_descripciones, model, top_n=5)

Juegos recomendados:
Era: The Empowered (Similaridad: 0.7521)
Pangenre RPG Core Rules (Revised 1st Edition) (Similaridad: 0.7456)
Plemię Ognistych Wilków (Similaridad: 0.7087)
MadWish (Similaridad: 0.7009)
Avalon Haunts 04 (Similaridad: 0.6965)


In [39]:
def recomendar_juegos_doc2vec_con_valoraciones(prompt, df_juegos, model, top_n=5, umbral_similaridad=0.1):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos usando los índices correctos del DataFrame
    vectors = [model.dv[idx] for idx in df_juegos.index]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Asegurarse de que las valoraciones coincidan con los juegos vectorizados
    valoraciones = df_juegos.loc[df_juegos.index.isin(df_juegos.index), 'Average_Rating'].fillna(0).values

    # Asegurar que los tamaños de cosine_similarities y valoraciones coincidan
    if len(cosine_similarities) != len(valoraciones):
        min_len = min(len(cosine_similarities), len(valoraciones))
        cosine_similarities = cosine_similarities[:min_len]
        valoraciones = valoraciones[:min_len]

    # Crear una métrica combinada (similaridad * valoraciones)
    similaridad_ponderada = cosine_similarities * valoraciones

    # Filtrar recomendaciones con similaridad negativa o muy baja
    indices_validos = [i for i, sim in enumerate(cosine_similarities) if sim > umbral_similaridad]

    # Ordenar los juegos con mayor similaridad ponderada
    indices_recomendados = sorted(indices_validos, key=lambda i: similaridad_ponderada[i], reverse=True)[:top_n]

    # Mostrar los juegos recomendados
    print("Juegos recomendados (con valoraciones y similaridad del coseno):")
    for index in indices_recomendados:
        print(f"{df_juegos.iloc[index]['Name']} (Similaridad ponderada: {similaridad_ponderada[index]:.4f}, "
              f"Similaridad del coseno: {cosine_similarities[index]:.4f}, "
              f"Valoración: {valoraciones[index]:.2f})")



In [40]:
prompt_usuario = 'I am looking for a game that is great for solo play and lasts about 30 minutes. I love games with a rich storytelling experience and immersive themes.'

In [41]:
# Ejecutar la recomendación usando Doc2Vec con valoraciones ponderadas y umbral
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1)

Juegos recomendados (con valoraciones y similaridad del coseno):
Punctuation Pirates and Their Grammar Galleons (Similaridad ponderada: 6.2393, Similaridad del coseno: 0.6239, Valoración: 10.00)
Touring Rock Band 2 (Similaridad ponderada: 5.8042, Similaridad del coseno: 0.6175, Valoración: 9.40)
The Way to Play (Similaridad ponderada: 5.6120, Similaridad del coseno: 0.7360, Valoración: 7.62)
OneDice Robin Hood (Similaridad ponderada: 5.4201, Similaridad del coseno: 0.6022, Valoración: 9.00)
Wings Over the World (Similaridad ponderada: 5.3775, Similaridad del coseno: 0.5377, Valoración: 10.00)


In [42]:
def recomendar_juegos_doc2vec_con_valoraciones(prompt, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos usando los índices correctos del DataFrame
    vectors = [model.dv[str(idx)] for idx in df_descripciones.index]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Asegurarse de que las valoraciones coincidan con los juegos vectorizados
    valoraciones = df_descripciones.loc[df_descripciones.index.isin(df_descripciones.index), 'Average_Rating'].fillna(0).values

    # Asegurar que los tamaños de cosine_similarities y valoraciones coincidan
    if len(cosine_similarities) != len(valoraciones):
        min_len = min(len(cosine_similarities), len(valoraciones))
        cosine_similarities = cosine_similarities[:min_len]
        valoraciones = valoraciones[:min_len]

    # Crear una métrica ponderada que priorice más la similaridad del coseno
    similaridad_ponderada = (cosine_similarities * w) + (valoraciones * (1 - w))

    # Filtrar recomendaciones con similaridad negativa o muy baja
    indices_validos = [i for i, sim in enumerate(cosine_similarities) if sim > umbral_similaridad]

    # Ordenar los juegos con mayor similaridad ponderada
    indices_recomendados = sorted(indices_validos, key=lambda i: similaridad_ponderada[i], reverse=True)[:top_n]

    # Mostrar los juegos recomendados
    print(f"Juegos recomendados (priorizando similaridad del coseno):")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad ponderada: {similaridad_ponderada[index]:.4f}, "
              f"Similaridad del coseno: {cosine_similarities[index]:.4f}, "
              f"Valoración: {valoraciones[index]:.2f})")



In [43]:
# Ejecutar la recomendación priorizando la similaridad del coseno (peso w = 0.7)
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7)

KeyError: "Key '0' not present"